## Data/Text Mining Project


Course: CISD 43<br>
Professor: Sohair Zaki<br>
Student: Fiona Xu and Jack Chen<br>
<br>
Purpose:<br>
- hello world
<br>

Details:<br>
1. Load libraries, data cleaning<br>
2. Exploratory Data Analysis (EDA)<br>
3. Deep Learning<br>
4. Conclusion<br>
5. References<br>

## Libraries, Functions, Load/Cleaning Data

In [58]:
# Parsing
import requests
from bs4 import BeautifulSoup
# Alpha Vantage API - stock data
from alpha_vantage.timeseries import TimeSeries
from alpha_vantage.fundamentaldata import FundamentalData
from alpha_vantage.cryptocurrencies import CryptoCurrencies
from alpha_vantage.techindicators import TechIndicators

# filter warnings
import warnings
warnings.filterwarnings("ignore")

# Data
import numpy as np
import pandas as pd

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline 

# Preprocessing
from sklearn.preprocessing import StandardScalar
from sklearn.model_selection import train_test_split

# Deep Learning
import tensorflow as tf

# Analyze Results
from sklearn.metrics import confusion_matrix, classification_report

### AlphaVantage API

In [ ]:
# Variable - store Alpha Vantage API key
# path =  r'C:\Users\Gumo\Desktop\Git\Notebook\keys\alphaVantage.txt'
# with open(path, mode='r') as w:
#     key = (w.readline())

In [ ]:
# URL Method
# def alpha(function, symbol):
#     url = 'https://www.alphavantage.co/query?function='+function+'&symbol='+symbol+'&apikey='+key
#     response = requests.get(url)

# Function AlphaVantage
def alpha(symbol, function, period=None, typ = None, tim=None):
    
    # Funciton - TimeSeries
    if function == 't':
        # Instance - TimeSeries Instance
        ts = TimeSeries(key = key, output_format='pandas')
        if period == 'd':
            data = ts.get_daily_adjusted(symbol, outputsize=outputsize)[0]
        elif period == 'w':
            data = ts.get_weekly_adjusted(symbol)[0]
        elif period == 'm':
            data = ts.get_monthly_adjusted(symbol)[0]

    # Funciton - FundamentalData
    elif function == 'f':
        # Instance - FundamentalData Instance
        fd = FundamentalData(key, output_format='pandas')
        if period == 'q':
            if typ == 'i':
                data = fd.get_income_statement_quarterly(symbol)[0]
                data.index = data.iloc[:,0]
                data=data.iloc[:,2:]
            elif typ == 'b':
                data = fd.get_balance_sheet_quarterly(symbol)[0]
                data.index = data.iloc[:,0]
                data=data.iloc[:,2:]
            elif typ=='c':
                data = fd.get_cash_flow_quarterly(symbol)[0]
                data.index = data.iloc[:,0]
                data=data.iloc[:,2:]
        elif period =='a':
            if typ == 'i':
                data = fd.get_income_statement_annual(symbol)[0]
                data.index = data.iloc[:,0]
                data=data.iloc[:,2:]
            elif typ == 'b':
                data = fd.get_balance_sheet_annual(symbol)[0]
                data.index = data.iloc[:,0]
                data=data.iloc[:,2:]
            elif typ=='c':
                data = fd.get_cash_flow_annual(symbol)[0]
                data.index = data.iloc[:,0]
                data=data.iloc[:,2:]
    
    # Funciton - TechnicalIndicator            
    elif function =='ti':
        ti = TechIndicators(key,output_format='pandas')

        # reassign period into TI format
        if period == 'm':
            period = 'monthly'
        elif period =='w':
            period = 'weekly'
        elif period == 'd':
            period ='daily'

        if typ == 'rsi':
            data=ti.get_rsi(symbol,interval=period,time_period=tim,series_type='close')[0]
        elif typ =='so':
            data = ti.get_stoch(symbol,interval=period)[0]
            
    # Funciton - CC
    elif function == 'c':
        # Instance - Crypto
        cc = CryptoCurrencies(key, output_format='pandas')
        pass
    return data

In [ ]:
# Variable Size
outputsize = 'compact'

# Variable Stock
symbol = 'cvx'

# Variable Function - t, f, c, ti (timeseries, financialdata, crypto, technical indicator)
function = 'ti'

# Variable Period
# TimeSeries: d,w,m,i (daily, weekly, monthly, interval(mins))
# Fundamental: q,a (quarterly,annually)
# Technical: d,w,m, (interval(mins))
period = 'm'

# Varible - Type
# Fundamental: i,b,c
# Technical: so,rsi
typ ='so'

# Varible - Time
tim = 60

info = alpha(symbol,function,period=period,typ = typ,tim=tim)
info.head()

In [ ]:
# save to local stock data
# info.to_csv('data/'+symbol.upper()+'stockPrice.csv')

### Functions

In [91]:
# Function to clean balance sheet df
def convMonth(x):
    if x == 'MAR':
        x = 3
    elif x == 'DEC':
        x = 12
    elif x =='JUN':
        x = 6
    elif x == 'SEP':
        x = 9
    return x 

### Load/Clean Data

In [140]:
# load dfs - data will be 01/2000 to 03/2022, 22 years and 3 month is 267 month, 
df_bs = pd.read_excel('data/df_cvx_bs_quarter.xlsx', engine='openpyxl')
df_is = pd.read_excel('data/df_cvx_is_quarter.xlsx', engine='openpyxl')
df_stock = pd.read_csv('data/CVXstockPrice.csv')
df_inflation = pd.read_csv('data/clean_inflation_rate.csv')
df_gas = pd.read_excel('data/gasPrices.xlsx', engine='openpyxl')


In [116]:
# we just need adjusted close , volumn
dfstock1 = df_stock.iloc[:,[0,5,6]]

# create year column and month column
dfstock1['year'] = df_stock.date.apply(lambda x : x.split('-')[0])
dfstock1['month'] = df_stock.date.apply(lambda x : x.split('-')[1])
dfstock1.tail()

# drop useless columns
dfstock2 = dfstock1.iloc[2:-1,1:]
# reset index
dfstock3 = dfstock2.reset_index(drop=True)

# convert month into int
dfstock3.month = dfstock3.month.apply(lambda x: int(x))

#assign to finaldf
dfstockf = dfstock3.sort_values(by=['year','month'])
dfstockf

,5. adjusted close,6. volume,year,month
266,18.2953,37687500.0,2000,1
265,16.4760,42961800.0,2000,2
264,20.3915,51176100.0,2000,3
263,18.7790,31502600.0,2000,4
262,20.5234,30692800.0,2000,5
...,...,...,...,...
4,110.7529,224686287.0,2021,11
3,115.1489,221128758.0,2021,12
2,128.8667,300049707.0,2022,1
1,142.7935,282783725.0,2022,2


#### bs df

In [33]:
# df_BalanceSheet - Clean Balance Sheet
# transpose df
df_bs1 = df_bs.T

# assign column names
df_bs1.columns = df_bs1.iloc[0,:]

# reset index and drop first row
df_bs2 = df_bs1.reset_index().iloc[1:,:]
df_bs2.head(3)

In [37]:
# see the columns
df_bs2.columns

Index(['index', 'Cash & Short-Term Investments', 'Cash Only',
       'Total Short Term Investments', 'Short-Term Receivables',
       'Accounts Receivables, Net', 'Accounts Receivables, Gross',
       'Bad Debt/Doubtful Accounts', 'Other Receivables', 'Inventories',
       'Finished Goods', 'Raw Materials', 'Other Current Assets',
       'Miscellaneous Current Assets', 'Total Current Assets',
       'Net Property, Plant & Equipment',
       'Property, Plant & Equipment - Gross',
       'Operating Lease Right-of-Use Assets', 'Accumulated Depreciation',
       'Total Long-Term Investments ', 'LT Investment - Affiliate Companies',
       'Other Long-Term Investments', 'Long-Term Note Receivable',
       'Intangible Assets', 'Goodwill', 'Deferred Tax Assets', 'Other Assets',
       'Tangible Other Assets', 'Total Assets',
       'Liabilities & Shareholders' Equity', 'ST Debt & Curr. Portion LT Debt',
       'Accounts Payable', 'Income Tax Payable', 'Other Current Liabilities',
       'Misc

In [100]:
# select the needed columns
df_bs3 = df_bs2[['index','Total Assets', 'Total Equity']].iloc[:89,:]
df_bs3.shape

(89, 3)

In [101]:
# since data is quarterly, we will append the data until we have 267 month
dfbs4 = df_bs3
dfbs5 = dfbs4.append(df_bs3)
dfbs6 = dfbs5.append(df_bs3)
dfbs6.shape

(267, 3)

In [102]:
# convert year and month
dfbs6['year'] = dfbs6['index'].apply(lambda x : int(x.split(' \'')[1]) + 2000)
dfbs6['month'] = dfbs6['index'].apply(lambda x : convMonth(x.split(' \'')[0]))
dfbs6.head()

Unnamed: 0,index,Total Assets,Total Equity,year,month
1,MAR '22,249.05,147.1,2022,3
2,DEC '21,239.54,139.94,2021,12
3,SEP '21,239.95,136.72,2021,9
4,JUN '21,242.81,133.91,2021,6
5,MAR '21,241.65,132.93,2021,3


In [110]:
dfbsf = dfbs6.sort_values(by=['year','month'])
dfbsf.shape

(267, 5)

#### is df

In [42]:
# Income Statement - Clean Balance Sheet
# transpose df
df_is1 = df_is.T

# assign column names
df_is1.columns = df_is1.iloc[0,:]

# reset index and drop first row
df_is2 = df_is1.reset_index().iloc[1:,:]
df_is2.head(3)

Unnamed: 0,index,Sales,Cost of Goods Sold (COGS) incl. D&A,COGS excluding D&A,Depreciation & Amortization Expense,Depreciation,Gross Income,SG&A Expense,Other SG&A,Other Operating Expense,...,EPS (diluted),Diluted Shares Outstanding,Total Shares Outstanding,Earnings Persistence,Dividends per Share,Payout Ratio,EBITDA,EBITDA,EBIT,Depreciation & Amortization Expense
1,MAR '22,53187.0,42045.0,38256.0,3789.0,3789.0,11142.0,1031.0,1031.0,2002.0,...,3.2188,1944.542,1964.813456,89.786,1.42,44.11582,NaN,11898.0,8109.0,3789.0
2,DEC '21,46207.0,37420.0,32958.0,4462.0,4462.0,8787.0,1357.0,1357.0,1779.0,...,2.63,1922.082,1929.806057,89.972,1.34,50.95057,NaN,10113.0,5651.0,4462.0
3,SEP '21,42349.0,33628.0,29288.0,4340.0,4340.0,8721.0,757.0,757.0,2075.0,...,3.181,1921.095,1927.685919,89.344,1.34,42.125118,NaN,10229.0,5889.0,4340.0


In [44]:
# see the columns
df_is2.columns

Index(['index', 'Sales', 'Cost of Goods Sold (COGS) incl. D&A',
       'COGS excluding D&A', 'Depreciation & Amortization Expense',
       'Depreciation', 'Gross Income', 'SG&A Expense', 'Other SG&A',
       'Other Operating Expense', 'EBIT (Operating Income)',
       'Nonoperating Income - Net', 'Equity in Earnings of Affiliates',
       'Other Income (Expense)', 'Interest Expense', 'Gross Interest Expense',
       'Interest Capitalized', 'Unusual Expense - Net', 'Impairments',
       'Property,Plant & Equipment', 'Financial Fixed Assets',
       'Restructuring Expense', 'Unrealized Valuation Gain/Loss',
       'Hedges/Derivatives', 'Excpl Chrgs - Others', 'Calamitous Events',
       'Pretax Income', 'Income Taxes', 'Equity in Earnings of Affiliates',
       'Consolidated Net Income', 'Minority Interest', 'Net Income',
       'Discontinued Operations', 'Net Income available to Common',
       'Per Share', 'EPS (recurring)', 'Basic Shares Outstanding',
       'Total Shares Outstanding'

In [83]:
# select needed columns
df_is3 = df_is2[['index','Sales','Net Income']].iloc[:89,:]
df_is3.shape

(89, 3)

In [86]:
# append df to 267 month
dfis4 = df_is3
dfis5 = dfis4.append(df_is3)
dfis6 = dfis5.append(df_is3)


In [87]:
# 22 year 3 month = 267 month
dfis6.shape

(267, 3)

In [95]:
# create year and month columns
dfis6['year'] = dfis6['index'].apply(lambda x : int(x.split(' \'')[1]) + 2000)
dfis6['month'] = dfis6['index'].apply(lambda x : convMonth(x.split(' \'')[0]))
dfis6.head()

Unnamed: 0,index,Sales,Net Income,year,month
1,MAR '22,53187.0,6259.0,2022,3
2,DEC '21,46207.0,5055.0,2021,12
3,SEP '21,42349.0,6111.0,2021,9
4,JUN '21,36385.0,3082.0,2021,6
5,MAR '21,31350.0,1377.0,2021,3


In [111]:
# sort by values
dfisf = dfis6.sort_values(by=['year','month'])
dfisf.shape

(267, 5)

In [144]:
# format inflation df
dfi1 = df_inflation.iloc[:267,:]
dfi1

,Unnamed: 0,Year,Month,Inflation Rate
0,22,2000,Jan,2.7
1,45,2000,Feb,3.2
2,68,2000,Mar,3.8
3,91,2000,Apr,3.1
4,114,2000,May,3.2
...,...,...,...,...
262,231,2021,Nov,6.8
263,254,2021,Dec,7.0
264,0,2022,Jan,7.5
265,23,2022,Feb,7.9


In [153]:
df_gas.Date = df_gas.Date.astype(str)

In [154]:
df_gas['year'] = df_gas.Date.apply(lambda x : x.split('-')[0])
df_gas['month'] = df_gas.Date.apply(lambda x : x.split('-')[1])
df_gas

,Date,U.S. All Grades All Formulations Retail Gasoline Prices (Dollars per Gallon),year,month
0,1993-04-15,1.078,1993,04
1,1993-05-15,1.100,1993,05
2,1993-06-15,1.097,1993,06
3,1993-07-15,1.078,1993,07
4,1993-08-15,1.062,1993,08
...,...,...,...,...
343,2021-11-15,3.491,2021,11
344,2021-12-15,3.406,2021,12
345,2022-01-15,3.413,2022,01
346,2022-02-15,3.611,2022,02


In [157]:
# get dfgas1
dfgas1 = df_gas.iloc[81:,:]

In [158]:
# create merged df
dfmerge = dfstockf

#rewrite quartly month into 12 month
dfmerge['sales'] = list(dfisf.Sales)
dfmerge['netIncome'] = list(dfisf['Net Income'])
dfmerge['Total Assets'] = list(dfbsf['Total Assets'])
dfmerge['Total Equity'] = list(dfbsf['Total Equity'])
dfmerge['inflation'] = list(dfi1['Inflation Rate'])
dfmerge['gas'] = list(dfgas1['U.S. All Grades All Formulations Retail Gasoline Prices (Dollars per Gallon)'])

# create the dfmerge1 reset index, rename columns
dfmerge1 = dfmerge.reset_index(drop=True).rename(columns={'5. adjusted close':'adjustedClose','6. volume':'volume','Total Assets': 'assets','Total Equity': 'equity'})

''' Units
sales - millions
netIncome - millions
asset - billions
equity - billions

'''

# assign new df
dfmerge2 = dfmerge1

# convert into millions
dfmerge2.equity = dfmerge1.equity * 1000
dfmerge2.assets = dfmerge1.assets * 1000
# convert into 3 month average
dfmerge2.sales = dfmerge1.sales/3
dfmerge2.netIncome = dfmerge1.netIncome/3
dfmerge2.year = dfmerge2.year.astype(int)
dfmerge2.inflation = dfmerge2.inflation.astype(float)

In [159]:
dfmerge2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 267 entries, 0 to 266
Data columns (total 10 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   adjustedClose  267 non-null    float64
 1   volume         267 non-null    float64
 2   year           267 non-null    int32  
 3   month          267 non-null    int64  
 4   sales          267 non-null    float64
 5   netIncome      267 non-null    float64
 6   assets         267 non-null    float64
 7   equity         267 non-null    float64
 8   inflation      267 non-null    float64
 9   gas            267 non-null    float64
dtypes: float64(8), int32(1), int64(1)
memory usage: 19.9 KB


In [161]:
dfmerge2.to_csv('data/final_df.csv')

## Exploratory Data Analysis (EDA)

#### Machine Learning

#### Conclusion


#### References


----------
CIS43 Project Link<br>
https://colab.research.google.com/drive/1OS0i9lEitkL7PTE7TpLuWVoqkjS0BPYC
<br>

Load local file onto colab<br>
https://www.youtube.com/watch?v=6HFlwqK3oeo
<br>
<br>

----------
Inflation Data<br>
https://www.usinflationcalculator.com/inflation/current-inflation-rates/#:~:text=The%20annual%20inflation%20rate%20for,at%208%3A30%20a.m.%20ET.
<br>
<br>
Gas Prices<br>
https://www.kaggle.com/datasets/mruanova/us-gasoline-and-diesel-retail-prices-19952021
<br>
<br>